# 時系列データの扱い

時系列データは、表の1行1行が独立ではなく、時間順でつながっています。
この順序を無視して分割や特徴量設計をすると、評価だけ高くて本番で崩れるモデルになりやすくなります。

このノートでは、売上の1ステップ先予測を題材に、観察、特徴量化、リーク回避、評価、将来予測までを一気通貫で確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


まず、上昇トレンド、季節性、キャンペーン効果を持つ疑似データを作ります。
実データがなくても、構造を入れた疑似データを作って小さく検証すると、何が効くのかを読み取りやすくなります。

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
rng = np.random.default_rng(7)

dates = pd.date_range("2018-01-01", periods=96, freq="MS")
trend = np.linspace(0, 95, len(dates))
seasonal = 18 * np.sin(2 * np.pi * np.arange(len(dates)) / 12)
promo = (np.arange(len(dates)) % 6 == 0).astype(int)
noise = rng.normal(loc=0.0, scale=4.5, size=len(dates))

sales = 220 + trend + seasonal + 16 * promo + noise

df = pd.DataFrame({
    "date": dates,
    "sales": sales,
    "promo": promo,
})

df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(11, 4.4))
ax.plot(df["date"], df["sales"], color="#2b6cb0", linewidth=2, label="sales")
ax.scatter(
    df.loc[df["promo"] == 1, "date"],
    df.loc[df["promo"] == 1, "sales"],
    color="#c05621",
    s=28,
    label="promo month",
)
ax.plot(df["date"], df["sales"].rolling(12).mean(), color="#4a5568", linewidth=1.6, label="12-month moving average")
ax.set_title("Monthly Sales")
ax.set_xlabel("date")
ax.set_ylabel("sales")
ax.legend(loc="upper left")
plt.tight_layout()
plt.show()


時系列予測で最重要なのは、「予測時点で使える情報だけ」で特徴量を作ることです。
ここでは `lag_1, lag_2, lag_3, lag_6, lag_12` と移動平均を作ります。
`lag_12` を使うため先頭12か月は欠損となり、`dropna()` で除外されます。これは正常な挙動です。

また、月を 1〜12 の整数のまま使うと 12月と1月が遠く見えるため、`sin/cos` で円環的な季節性を表します。
移動平均には `shift(1)` を入れ、当月値の混入を防いでリークを避けます。

In [ ]:
def make_time_features(frame: pd.DataFrame, lag_steps=(1, 2, 3, 6, 12), roll_windows=(3, 6)) -> pd.DataFrame:
    out = frame.sort_values("date").reset_index(drop=True).copy()

    for lag in lag_steps:
        out[f"lag_{lag}"] = out["sales"].shift(lag)

    for window in roll_windows:
        out[f"roll_mean_{window}"] = out["sales"].shift(1).rolling(window=window).mean()

    month = out["date"].dt.month
    out["month_sin"] = np.sin(2 * np.pi * month / 12)
    out["month_cos"] = np.cos(2 * np.pi * month / 12)

    return out.dropna().reset_index(drop=True)

feature_df = make_time_features(df)
feature_df.head()


In [ ]:
feature_cols = [
    "promo",
    "month_sin",
    "month_cos",
    "lag_1",
    "lag_2",
    "lag_3",
    "lag_6",
    "lag_12",
    "roll_mean_3",
    "roll_mean_6",
]

print("raw period :", df["date"].min().date(), "to", df["date"].max().date(), "rows:", len(df))
print("feat period:", feature_df["date"].min().date(), "to", feature_df["date"].max().date(), "rows:", len(feature_df))
print("features:", feature_cols)


自己相関を見ると、「何か月前の値と似た動きをするか」が分かります。
季節周期の候補やラグの当たりを付けるときに役立ちます。

In [ ]:
def autocorr_at_lag(values: np.ndarray, lag: int) -> float:
    x = values[lag:]
    y = values[:-lag]
    if x.std() == 0 or y.std() == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

series = feature_df["sales"].to_numpy()
lags = np.arange(1, 25)
autocorr_values = np.array([autocorr_at_lag(series, int(lag)) for lag in lags])

fig, ax = plt.subplots(figsize=(10, 3.6))
ax.bar(lags, autocorr_values, color="#2f855a")
ax.axhline(0, color="#1a202c", linewidth=1)
ax.set_title("Autocorrelation by lag")
ax.set_xlabel("lag")
ax.set_ylabel("corr")
plt.tight_layout()
plt.show()


ここからは、最後の12か月をテストに固定して評価します。
まずは基準として、1か月前の値をそのまま予測に使う単純ベースラインを作ります。

In [ ]:
test_horizon = 12
train_df = feature_df.iloc[:-test_horizon].copy()
test_df = feature_df.iloc[-test_horizon:].copy()

X_train = train_df[feature_cols]
y_train = train_df["sales"]
X_test = test_df[feature_cols]
y_test = test_df["sales"]

naive_pred = test_df["lag_1"].to_numpy()
naive_mae = mean_absolute_error(y_test, naive_pred)
print(f"naive MAE: {naive_mae:.3f}")


対照として、やってはいけないランダム分割も試します。
ランダム分割では「後年のデータを学習に使いながら前の年をテストする」状態が起き、本番では使えない未来情報が暗黙に混ざります。

In [ ]:
X_all = feature_df[feature_cols]
y_all = feature_df["sales"]

X_tr_rand, X_te_rand, y_tr_rand, y_te_rand = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42, shuffle=True
)

rand_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0)),
])
rand_pipeline.fit(X_tr_rand, y_tr_rand)
rand_pred = rand_pipeline.predict(X_te_rand)
rand_mae = mean_absolute_error(y_te_rand, rand_pred)

print(f"random split ridge MAE (invalid for TS): {rand_mae:.3f}")


次に、時系列順を守った1ステップ先予測（予測のたびに実測で更新する前提）でモデルを比較します。
線形モデルは解釈しやすく、木モデルは非線形性を拾いやすいという違いがあります。

In [ ]:
ridge_model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0)),
])

rf_model = RandomForestRegressor(
    n_estimators=350,
    max_depth=8,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
)

ridge_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

ridge_pred = ridge_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

ridge_mae = mean_absolute_error(y_test, ridge_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)

print(f"naive MAE                : {naive_mae:.3f}")
print(f"chronological ridge MAE  : {ridge_mae:.3f}")
print(f"chronological rf MAE     : {rf_mae:.3f}")
print(f"random split ridge MAE   : {rand_mae:.3f} (invalid setup)")


In [ ]:
one_step_df = test_df[["date", "sales"]].copy()
one_step_df["naive"] = naive_pred
one_step_df["ridge"] = ridge_pred
one_step_df["rf"] = rf_pred

fig, ax = plt.subplots(figsize=(11, 4.4))
ax.plot(one_step_df["date"], one_step_df["sales"], marker="o", linewidth=2, label="actual")
ax.plot(one_step_df["date"], one_step_df["naive"], marker="o", linewidth=1.5, label="naive")
ax.plot(one_step_df["date"], one_step_df["ridge"], marker="o", linewidth=1.5, label="ridge")
ax.plot(one_step_df["date"], one_step_df["rf"], marker="o", linewidth=1.5, label="random forest")
ax.set_title("One-Step Forecast (Observed Updates)")
ax.set_xlabel("date")
ax.set_ylabel("sales")
ax.legend()
plt.tight_layout()
plt.show()


固定起点で12か月先まで当てるには、逐次予測のバックテストが必要です。
ここでは学習期間を固定し、予測値を次月ラグへ入れながら12ステップ先まで進めます。
`promo` は「6か月ごとに実施される既知の計画値」を使える前提にしています。

In [ ]:
def predict_next_months(history: pd.DataFrame, model, steps: int = 12) -> pd.DataFrame:
    working = history.sort_values("date").reset_index(drop=True).copy()
    preds = []

    for _ in range(steps):
        next_date = (working["date"].iloc[-1] + pd.offsets.MonthBegin(1)).normalize()
        month_index = len(working)
        next_promo = int(month_index % 6 == 0)

        next_row = {
            "date": next_date,
            "promo": next_promo,
            "lag_1": float(working["sales"].iloc[-1]),
            "lag_2": float(working["sales"].iloc[-2]),
            "lag_3": float(working["sales"].iloc[-3]),
            "lag_6": float(working["sales"].iloc[-6]),
            "lag_12": float(working["sales"].iloc[-12]),
            "roll_mean_3": float(working["sales"].iloc[-3:].mean()),
            "roll_mean_6": float(working["sales"].iloc[-6:].mean()),
            "month_sin": float(np.sin(2 * np.pi * next_date.month / 12)),
            "month_cos": float(np.cos(2 * np.pi * next_date.month / 12)),
        }

        row_df = pd.DataFrame([next_row])
        y_hat = float(model.predict(row_df[feature_cols])[0])

        preds.append({"date": next_date, "forecast": y_hat})
        working = pd.concat(
            [working, pd.DataFrame([{"date": next_date, "sales": y_hat, "promo": next_promo}])],
            ignore_index=True,
        )

    return pd.DataFrame(preds)

train_raw = df.iloc[:-test_horizon].copy()
test_raw = df.iloc[-test_horizon:].copy()
train_raw_feat = make_time_features(train_raw)

ridge_recursive = clone(ridge_model)
rf_recursive = clone(rf_model)
ridge_recursive.fit(train_raw_feat[feature_cols], train_raw_feat["sales"])
rf_recursive.fit(train_raw_feat[feature_cols], train_raw_feat["sales"])

ridge_backtest = predict_next_months(train_raw, ridge_recursive, steps=test_horizon)
rf_backtest = predict_next_months(train_raw, rf_recursive, steps=test_horizon)

ridge_recursive_mae = mean_absolute_error(test_raw["sales"], ridge_backtest["forecast"])
rf_recursive_mae = mean_absolute_error(test_raw["sales"], rf_backtest["forecast"])

print(f"recursive ridge MAE (12-step): {ridge_recursive_mae:.3f}")
print(f"recursive rf MAE (12-step)   : {rf_recursive_mae:.3f}")


In [ ]:
recursive_plot = test_raw[["date", "sales"]].copy()
recursive_plot["ridge_recursive"] = ridge_backtest["forecast"].to_numpy()
recursive_plot["rf_recursive"] = rf_backtest["forecast"].to_numpy()

fig, ax = plt.subplots(figsize=(11, 4.4))
ax.plot(recursive_plot["date"], recursive_plot["sales"], marker="o", linewidth=2, label="actual")
ax.plot(recursive_plot["date"], recursive_plot["ridge_recursive"], marker="o", linewidth=1.5, label="ridge recursive")
ax.plot(recursive_plot["date"], recursive_plot["rf_recursive"], marker="o", linewidth=1.5, label="rf recursive")
ax.set_title("Recursive 12-Step Backtest")
ax.set_xlabel("date")
ax.set_ylabel("sales")
ax.legend()
plt.tight_layout()
plt.show()


分割1回だけでは偶然の影響が残るため、`TimeSeriesSplit` でウォークフォワード評価も確認します。
この評価では常に「過去で学習し未来を検証する」順序が守られます。

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(
    ridge_model,
    X_all,
    y_all,
    cv=tscv,
    scoring="neg_mean_absolute_error",
)

cv_mae = -cv_scores
print("TimeSeriesSplit MAE:", np.round(cv_mae, 3))
print("mean:", round(cv_mae.mean(), 3), "std:", round(cv_mae.std(), 3))


最後に運用想定として、利用可能な全期間で再学習して未来12か月を予測します。

In [ ]:
ridge_final = clone(ridge_model)
ridge_final.fit(X_all, y_all)

future_pred = predict_next_months(df, ridge_final, steps=12)
future_pred.head()


In [ ]:
recent_hist = df.tail(24)
fig, ax = plt.subplots(figsize=(11, 4.4))
ax.plot(recent_hist["date"], recent_hist["sales"], marker="o", label="history (last 24 months)")
ax.plot(future_pred["date"], future_pred["forecast"], marker="o", label="future forecast (ridge)")
ax.set_title("History and 12-Month Forecast")
ax.set_xlabel("date")
ax.set_ylabel("sales")
ax.legend()
plt.tight_layout()
plt.show()


時系列モデリングでは、モデルを複雑にする前に「時間順を壊していないか」を必ず確認してください。
ラグ特徴と移動平均のような基本設計でも、評価設計が正しければ実務で使える強い基準線になります。